In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [2]:
model_args = ClassificationArgs(num_train_epochs=4)
model_args.learning_rate = 2e-5
model_args.max_seq_length=512
model_args.sliding_window=True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.eval_batch_size = 32
model_args.train_batch_size =32
model_args.use_cached_eval_features = True
model_args.use_early_stopping = True
model_args.early_stopping_metric = 'eval_loss'
model_args.early_stopping_metric_minimize = True
model_args.save_steps = -1
model_args.save_model_every_epoch = False
model_args.reprocess_input_data = True
model_args.do_lower_case = True
model_args.manual_seed = 42

In [3]:
model = ClassificationModel(
    "distilbert", "outputs/best_model/",num_labels=3
)

In [4]:
predictions, raw_outputs=model.predict(["I like the product"])

In [5]:
predictions[0]

2

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('data/big_corpus.csv',index_col=0)
df['reviews']= df['reviews'].astype(str)

In [8]:
df.head()

,ratings,reviews
364850,5,Love this 3ds game. I made a zombie Togepi wit...
216628,5,Fun game
319038,5,This game is so fun. I think it is the best S...
440696,5,I hated the first game but absolutely love thi...
287513,5,Love the Tales games. Glad I purchased this one.


In [9]:
from sklearn.utils import shuffle
df = shuffle(df)

In [10]:
df.dropna(subset = ["reviews"], inplace=True)

In [11]:
from nltk.tokenize import TweetTokenizer
from emoji import demojize
import re

tokenizer = TweetTokenizer()

def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())

In [12]:
import re
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [13]:
df['text'] = df['reviews'].apply(lambda x:clean_text(normalizeTweet(x)))

In [14]:
def score_to_Target(value):
    if value >= 5:
        return 2
    if value <= 4 and value >= 2:
        return 1
    else:
        return 0

In [15]:
df['labels'] = df['ratings'].apply(lambda x:score_to_Target(x))

In [26]:
def prediction_to_class(text):
    predictions, raw_outputs=model.predict(text)
    return predictions

In [27]:
list_of_str = list(df['text'])

In [28]:
predictions = prediction_to_class(list_of_str)

Token indices sequence length is longer than the specified maximum sequence length for this model (691 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (831 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (629 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (609 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (864 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [34]:
df['predicted'] = predictions

In [35]:
df.head()

,ratings,reviews,text,labels,predicated,predicted
219006,5,great!,great,2,2,2
480121,3,So I really really wanted to like this game an...,so i really really wanted to like this game an...,1,1,1
221474,1,didn't work,did nt work,0,0,0
439944,5,:),,2,2,2
107735,5,If you are into FPS. I would check this title...,if you are into fps i would check this title ...,2,2,2


In [36]:
rating_classes = list(df["labels"])
predicated_values  = list(df["predicted"])
target_names = ["negative", "neutral", "positive"]
from sklearn.metrics import classification_report
print(classification_report(rating_classes, predicated_values, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.42      0.82      0.56      6238
     neutral       0.63      0.58      0.60     33554
    positive       0.84      0.80      0.82     60208

    accuracy                           0.72    100000
   macro avg       0.63      0.73      0.66    100000
weighted avg       0.74      0.72      0.73    100000



In [37]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [38]:
plot_confusion_matrix(cm           = cm,
                      normalize    = False,
                      target_names = ["negative", "neutral", "positive"],
                      title        = "Confusion Matrix")

NameError: name 'cm' is not defined